Pipeline is a powerful tool to standardise your operations and chain then in a sequence, make unions and finetune parameters. In this example we will:
* create a simple pipeline of default sklearn estimators/transformers
* create our own estimator/transformer
* create a pipeline which will process features in a different way and then join them horizontally
* finetune some parameters

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv(r'D:\Machine Learning\Deployment of Machine Learning Models\Section 4 Building a Reproducible Machine Learning Pipeline\sklearn Pipeline\Data\jigsaw-toxic-comment-classification-challenge\train.csv')
x = df['comment_text'].values[:5000]
y = df['toxic'].values[:5000]

In [3]:
# default params
scoring='roc_auc'
cv=3
n_jobs=-1
max_features = 2500

Simple pipelines of default sklearn TfidfVectorizer to prepare features and Logistic Reegression to make predictions. 

In [4]:
tfidf = TfidfVectorizer(max_features=max_features)
lr = LogisticRegression()
p = Pipeline([
    ('tfidf', tfidf),
    ('lr', lr)
])

cross_val_score(estimator=p, X=x, y=y, scoring=scoring, cv=cv, n_jobs=n_jobs)

array([0.91248134, 0.92889109, 0.92437274])

Lets create or own Estimator to reproduce Jeremy`s notebook in pipelines. This estimator is created with sklearn BaseEstimator class and needs to have fit and transform methods. First Pipeline calls fit methods to learn your dataset and then calls transform to apply knowledge and does some transformations.

In [5]:
class NBFeaturer(BaseEstimator):
    def __init__(self, alpha):
        self.alpha = alpha
    
    def preprocess_x(self, x, r):
        return x.multiply(r)
    
    def pr(self, x, y_i, y):
        p = x[y==y_i].sum(0)
        return (p+self.alpha) / ((y==y_i).sum()+self.alpha)

    def fit(self, x, y=None):
        self._r = sparse.csr_matrix(np.log(self.pr(x,1,y) / self.pr(x,0,y)))
        return self
    
    def transform(self, x):
        x_nb = self.preprocess_x(x, self._r)
        return x_nb

In [6]:
tfidf = TfidfVectorizer(max_features=max_features)
lr = LogisticRegression()
nb = NBFeaturer(1)
p = Pipeline([
    ('tfidf', tfidf),
    ('nb', nb),
    ('lr', lr)
])

cross_val_score(estimator=p, X=x, y=y, scoring=scoring, cv=cv, n_jobs=n_jobs)

array([0.91851711, 0.93572898, 0.91808511])

Lets add one more custom Estimator to our pipeline, called Lemmatizer

In [7]:
class Lemmatizer(BaseEstimator):
    def __init__(self):
        self.l = WordNetLemmatizer()
        
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = map(lambda r:  ' '.join([self.l.lemmatize(i.lower()) for i in r.split()]), x)
        x = np.array(list(x))
        return x

In [8]:
lm = Lemmatizer()
tfidf = TfidfVectorizer(max_features=max_features)
lr = LogisticRegression()
nb = NBFeaturer(1)
p = Pipeline([
    ('lm', lm),
    ('tfidf', tfidf),
    ('nb', nb),
    ('lr', lr)
])

cross_val_score(estimator=p, X=x, y=y, scoring=scoring, cv=cv, n_jobs=n_jobs)

array([0.9267607 , 0.93755361, 0.91863238])

Pipelines also allow you to process different features in a different way and then concat the result. FeatureUnion halps us with this. Lets create additional tfidf vectorizer for chars and join its results with words vectorizer.

In [9]:
max_features = 2500
lm = Lemmatizer()
tfidf_w = TfidfVectorizer(max_features=max_features, analyzer='word')
tfidf_c = TfidfVectorizer(max_features=max_features, analyzer='char')
lr = LogisticRegression()
nb = NBFeaturer(1)
p = Pipeline([
    ('lm', lm),
    ('wc_tfidfs', 
         FeatureUnion([
            ('tfidf_w', tfidf_w), 
            ('tfidf_c', tfidf_c), 
         ])
    ),
    ('nb', nb),
    ('lr', lr)
])

cross_val_score(estimator=p, X=x, y=y, scoring=scoring, cv=cv, n_jobs=n_jobs)

array([0.94049207, 0.94488786, 0.93058846])

Who does not like finetuning? Lets make it simple with pipelines  and GridSearchCV/RandomizedSearchCV. 

In [10]:
param_grid = [{
    'wc_tfidfs__tfidf_w__max_features': [2500], 
    'wc_tfidfs__tfidf_c__stop_words': [2500, 5000],
    'lr__C': [3.],
}]

grid = GridSearchCV(p, cv=cv, n_jobs=n_jobs, param_grid=param_grid, scoring=scoring, 
                            return_train_score=False, verbose=1)
grid.fit(x, y)
grid.cv_results_

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    4.7s finished
C:\Users\puabh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'mean_fit_time': array([3.09293946, 2.10390925]),
 'std_fit_time': array([0.76640666, 0.12394552]),
 'mean_score_time': array([0.74468255, 0.92918086]),
 'std_score_time': array([0.20834602, 0.04157243]),
 'param_lr__C': masked_array(data=[3.0, 3.0],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_wc_tfidfs__tfidf_c__stop_words': masked_array(data=[2500, 5000],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_wc_tfidfs__tfidf_w__max_features': masked_array(data=[2500, 2500],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'lr__C': 3.0,
   'wc_tfidfs__tfidf_c__stop_words': 2500,
   'wc_tfidfs__tfidf_w__max_features': 2500},
  {'lr__C': 3.0,
   'wc_tfidfs__tfidf_c__stop_words': 5000,
   'wc_tfidfs__tfidf_w__max_features': 2500}],
 'split0_test_score': array([0.93779186, 0.93779186]),
 'split1_test_score': array([0.9408574, 0.9408574]),
 'split2

In [11]:
param_grid = [{
    'wc_tfidfs__tfidf_w__max_features': [2500, 5000, 10000], 
    'wc_tfidfs__tfidf_c__stop_words': [2500, 5000, 10000],
    'lr__C': [1., 3., 4.],
}]

grid = RandomizedSearchCV(p, cv=cv, n_jobs=n_jobs, param_distributions=param_grid[0], n_iter=1, 
                          scoring=scoring, return_train_score=False, verbose=1)
grid.fit(x, y)
grid.cv_results_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.4s finished
C:\Users\puabh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'mean_fit_time': array([1.62332527]),
 'std_fit_time': array([0.08461147]),
 'mean_score_time': array([0.70545284]),
 'std_score_time': array([0.08234663]),
 'param_wc_tfidfs__tfidf_w__max_features': masked_array(data=[10000],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_wc_tfidfs__tfidf_c__stop_words': masked_array(data=[5000],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_lr__C': masked_array(data=[1.0],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'wc_tfidfs__tfidf_w__max_features': 10000,
   'wc_tfidfs__tfidf_c__stop_words': 5000,
   'lr__C': 1.0}],
 'split0_test_score': array([0.95026446]),
 'split1_test_score': array([0.95762254]),
 'split2_test_score': array([0.94396264]),
 'mean_test_score': array([0.95061788]),
 'std_test_score': array([0.00558195]),
 'rank_test_score': array([1])}

Useful links:
* http://scikit-learn.org/stable/modules/pipeline.html#pipeline
* https://github.com/scikit-learn-contrib/project-template